- We loose structure of dialogue using the tokenizer in preprocess_data

- Emotions encoded using MultiLabelBinarizer doesn't tell us
  anymore how many times a single emotion is present in the dialogue
  and where it is

In [1]:
!pip install torch==1.13.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install transformers==4.30.0
!pip install datasets==2.13.2
!pip install accelerate -U
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 348.1 kB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
# system packages
from pathlib import Path
import shutil
import urllib
import tarfile
import sys
import os
# data and numerical management packages
import pandas as pd
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
# useful during debugging (progress bars)
from tqdm import tqdm
from transformers import set_seed

seed = 852

random.seed(seed)
np.random.seed(seed)
set_seed(seed)

In [10]:
dataset_folder = Path.cwd().joinpath("Datasets/MELD_train_efr.json")
#dataset_path = dataset_folder.joinpath('/MELD_train_efr.json')
df = pd.read_json(dataset_folder)
#df['triggers'] = df['triggers'].fillna(value=0, inplace=False)#.replace('None', 0.0)

In [11]:
triggers = df['triggers']
for row in range(triggers.shape[0]):
    for trigger in range(len(triggers[row])):
        if triggers[row][trigger] == None:
            triggers[row][trigger] = 0.0

df['triggers'] = triggers

In [12]:
emotions = df['emotions'].explode().unique()
emotions

array(['neutral', 'surprise', 'fear', 'sadness', 'joy', 'disgust',
       'anger'], dtype=object)

In [13]:
triggers = df['triggers'].explode().unique()
triggers

array([0.0, 1.0], dtype=object)

In [14]:
dialogues = df['utterances']
#print(sentences)
max_len_utterances = 0
index = 0
for idx, dialogue in enumerate(dialogues):
  if len(dialogue) > max_len_utterances:
    max_len_utterances = len(dialogue)
    index = idx
max_len_utterances,index

(24, 219)

In [62]:
dialogues = df['utterances']
#print(sentences)
max_len_sentence = 0
index = 0
for idx, dialogue in enumerate(dialogues):
  for idx_sentence, utterance in enumerate(dialogue):
    if len(utterance.split()) > max_len_sentence:
      max_len_sentence = len(utterance.split())
      index = idx,idx_sentence
max_len_sentence,index

(69, (1675, 11))

In [245]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
model_card = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_card)
model = AutoModelForSequenceClassification.from_pretrained(model_card,
                                                           num_labels=24,#len(emotions)+1, # +1 padding
                                                           problem_type="multi_label_classification")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

### Be careful for the token used during padding

In [23]:
padded_dialogues = [seq + ['[PAD]'] * (max_len_utterances - len(seq)) for seq in dialogues]

In [24]:
padded_emotions = [seq + ['[PAD]'] * (max_len_utterances - len(seq)) for seq in df['emotions']]

In [25]:
triggers = df['triggers']
padded_triggers = [seq + [0] * (max_len_utterances - len(seq)) for seq in triggers]

In [44]:
tokenized_input_ids = []
tokenized_attention_mask = []
tokenized_token_type_ids = []
i = 0
for utterance_list in padded_dialogues:
  utterance_input_ids = []
  utterance_attention_mask = []
  utterance_token_type_ids = []
  for utterance in utterance_list:
    tokenized_texts = tokenizer(utterance,return_tensors='pt')# padding='max_length', max_length=max_len, )
    utterance_input_ids.extend(tokenized_texts.input_ids)
    utterance_attention_mask.extend(tokenized_texts.attention_mask)
    utterance_token_type_ids.extend(tokenized_texts.token_type_ids)
"""    if i == 0:
        print(utterance)
        print(utterance_input_ids)
        print(tokenized_input_ids)
        i+=1"""
  tokenized_input_ids.append(utterance_input_ids)
  tokenized_attention_mask.append(utterance_attention_mask)
  tokenized_token_type_ids.append(utterance_token_type_ids)


also I was the point person on my company's transition from the KL-5 to GR-6 system.
[tensor([  101,  2036,  1045,  2001,  1996,  2391,  2711,  2006,  2026,  2194,
         1005,  1055,  6653,  2013,  1996,  1047,  2140,  1011,  1019,  2000,
        24665,  1011,  1020,  2291,  1012,   102])]
[]


In [27]:
emotions_input_ids = []
emotions_attention_mask = []
emotions_token_type_ids = []

for utterance_list in padded_emotions:
  emotion_input_ids = []
  emotion_attention_mask = []
  emotion_token_type_ids = []
  for utterance in utterance_list:
    tokenized_texts = tokenizer(utterance,return_tensors='pt')# padding='max_length', max_length=max_len, )
    emotion_input_ids.append(tokenized_texts.input_ids)
    emotion_attention_mask.append(tokenized_texts.attention_mask)
    emotion_token_type_ids.append(tokenized_texts.token_type_ids)

  emotions_input_ids.append(emotion_input_ids)
  emotions_attention_mask.append(emotion_attention_mask)
  emotions_token_type_ids.append(emotion_token_type_ids)

In [28]:
df['emotions'][:5]

0       [neutral, neutral, neutral, neutral, surprise]
1    [neutral, neutral, neutral, neutral, surprise,...
2    [neutral, neutral, neutral, neutral, surprise,...
3    [neutral, neutral, neutral, neutral, surprise,...
4                  [surprise, sadness, surprise, fear]
Name: emotions, dtype: object

In [ ]:
"""
pad = torch.from_numpy(np.array([0]))
padded_utterances_input_ids = [seq + [pad] * (max_len - len(seq)) for seq in tokenized_input_ids]
padded_utterances_attention_mask = [seq + [torch.from_numpy(np.array([1]))] * (max_len - len(seq)) for seq in tokenized_attention_mask]
padded_utterances_token_type_ids = [seq + [pad] * (max_len - len(seq)) for seq in tokenized_token_type_ids]
pad_as_1 = torch.from_numpy(np.array([1]))
padded_emotions_input_ids = [seq + [pad] * (max_len - len(seq)) for seq in emotions_input_ids]
padded_emotions_attention_mask = [seq + [pad_as_1] * (max_len - len(seq)) for seq in emotions_attention_mask]
padded_emotions_token_type_ids = [seq + [pad] * (max_len - len(seq)) for seq in emotions_token_type_ids]
"""

In [169]:
emotion_mapping = {emotion: idx+1 for idx, emotion in enumerate(emotions)}
emotions_numerical_format = [[emotion_mapping[emotion] for emotion in dialogue] for dialogue in df['emotions']]
padded_emotions = [seq + [0] * (max_len_utterances - len(seq)) for seq in emotions_numerical_format]

In [212]:
labels_np = np.array(padded_emotions, dtype=int)
labels_np[0.8*4000: 0.8*4000+400]

In [223]:
data = {
    'episode' : df['episode'],
    'labels' : labels_np.tolist(),#emotions_input_ids, #tokenized emotions
    'emotions' : padded_emotions,#emotions_input_ids, #tokenized emotions
    'utterances': padded_dialogues,
}
df_tokenized = pd.DataFrame(data)
from sklearn.model_selection import train_test_split
train_data, temp_data = train_test_split(df_tokenized, train_size=0.8, shuffle=False)
val_data, test_data = train_test_split(temp_data, test_size=0.5, shuffle=False)
train_data.shape, val_data.shape,test_data.shape

((3200, 4), (400, 4), (400, 4))

In [211]:
0.8*4000: 0.8*4000+400

3200.0

In [137]:
from datasets import Dataset
training_hf = Dataset.from_pandas(train_data)
validation_hf = Dataset.from_pandas(val_data)
test_hf = Dataset.from_pandas(test_data)

In [171]:
train_data['utterances']

0       [also I was the point person on my company's t...
1       [also I was the point person on my company's t...
2       [also I was the point person on my company's t...
3       [also I was the point person on my company's t...
4       [But then who? The waitress I went out with la...
                              ...                        
3195    [I can't believe this. Do you think that your ...
3196    [I can't believe this. Do you think that your ...
3197    [Dr. Geller, there's a seat over here., Thank ...
3198    [Dr. Geller, there's a seat over here., Thank ...
3199    [Dr. Geller, there's a seat over here., Thank ...
Name: utterances, Length: 3200, dtype: object

In [239]:
def preprocess_text(texts):
    tokenized_inputs = tokenizer([" ".join(utterance) for utterance in texts['utterances']],padding='max_length', truncation=True, max_length=max_len_sentence, return_tensors='pt')

    labels_tensor = torch.tensor(labels_np[3200:3600,:], dtype=torch.float)
    return Dataset.from_dict({
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels': labels_tensor
    })
"""def preprocess_data(data, tokenizer, mlb):
    # Tokenize the utterances
    tokenized_inputs = tokenizer(
        [" ".join(utterance) for utterance in data['utterances']],
        padding='max_length',
        truncation=True,
        max_length=128, # we might wanna change this to max_len !
        return_tensors='pt'
    )

    # Encode the emotions
    encoded_labels = mlb.transform(data['emotions'])
    # Convert labels to tensors
    labels_tensor = torch.tensor(encoded_labels, dtype=torch.float)
    # Ensure the encoded labels are the correct shape (this step is crucial to avoid the batch size mismatch)
    if labels_tensor.shape[0] != tokenized_inputs['input_ids'].shape[0]:
        raise ValueError(f"Number of examples {tokenized_inputs['input_ids'].shape[0]} does not match number of labels {labels_tensor.shape[0]}")

    return Dataset.from_dict({
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels': labels_tensor
    })"""

# Now call preprocess_data to process your train and validation data
#train_dataset = preprocess_text(train_data)
val_dataset = preprocess_text(val_data)

"""training = training_hf.map(preprocess_text, batched=True)
validation = validation_hf.map(preprocess_text, batched=True)
test = test_hf.map(preprocess_text, batched=True)"""

'training = training_hf.map(preprocess_text, batched=True)\nvalidation = validation_hf.map(preprocess_text, batched=True)\ntest = test_hf.map(preprocess_text, batched=True)'

In [253]:
print(val_dataset['input_ids'][0])

[101, 2852, 1012, 21500, 3917, 1010, 2045, 1005, 1055, 1037, 2835, 2058, 2182, 1012, 4067, 2017, 1010, 2852, 1012, 8109, 1010, 2021, 1045, 1005, 1049, 2383, 2026, 6265, 2012, 2023, 2795, 1010, 2182, 1999, 1996, 2690, 1012, 1045, 1005, 1049, 2383, 6265, 2157, 2182, 1010, 2007, 2026, 2204, 2767, 9558, 1010, 2065, 2002, 1005, 2222, 4133, 2007, 2033, 1012, 1045, 2097, 4133, 2007, 2017, 2852, 1012, 21500, 3917, 102]


In [142]:
for name, param in model.named_parameters():
    if 'embeddings' in name: # Layer names not containing 'classifier' will be frozen
        param.requires_grad = False

In [246]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test_dir",                 # where to save model
    learning_rate=1e-5,
    per_device_train_batch_size=8,         # accelerate defines distributed training
    per_device_eval_batch_size=50,
    num_train_epochs=1,
    weight_decay=0.1,
    evaluation_strategy="epoch",           # when to report evaluation metrics/losses
    logging_strategy="epoch",
    save_strategy="epoch",                 # when to save checkpoint
    load_best_model_at_end=True,
    report_to='none'                       # disabling wandb (default)
)
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=val_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.441000,0.264807


TrainOutput(global_step=50, training_loss=0.440987548828125, metrics={'train_runtime': 13.7165, 'train_samples_per_second': 29.162, 'train_steps_per_second': 3.645, 'total_flos': 14186131948800.0, 'train_loss': 0.440987548828125, 'epoch': 1.0})

In [247]:
test_prediction_info = trainer.predict(val_dataset)
#metrics = compute_metrics(test_prediction_info)
#print(test_prediction_info)
test_prediction_info.predictions[0],test_prediction_info.label_ids[0]

(array([ 0.15827195,  0.44049922,  0.5801743 ,  1.365504  ,  0.05035927,
         1.0560385 ,  0.3032352 , -0.23196907,  0.6998879 ,  0.2703544 ,
        -0.24004805,  0.4625907 , -0.6529549 ,  0.52764946, -0.5102945 ,
        -0.36305633, -0.5296073 , -0.20032535,  0.3740353 , -0.26822382,
        -0.4213328 , -0.44511944,  0.06515053, -1.1275369 ], dtype=float32),
 array([1., 1., 1., 1., 1., 4., 3., 1., 5., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.], dtype=float32))

In [250]:
val_data['emotions'][3200]

[1, 1, 1, 1, 1, 4, 3, 1, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

### Nalin's scribbles below

In [147]:
from sklearn.metrics import f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Apply sigmoid to logits and threshold to get multi-label predictions
    sigmoid_logits = 1 / (1 + np.exp(-logits))  # Sigmoid function
    threshold = 0.5
    predictions = (sigmoid_logits > threshold).astype(int)

    # Calculate F1 score for each individual label/class
    f1_scores = [f1_score(labels[:, i], predictions[:, i], average='binary') for i in range(num_labels)]

    # Sequence F1: compute the F1-score for each dialogue and report the average score
    sequence_f1 = np.mean(f1_scores)

    # Unrolled Sequence F1: flatten all utterances and compute the F1-score
    unrolled_labels = labels.flatten()
    unrolled_predictions = predictions.flatten()
    unrolled_sequence_f1 = f1_score(unrolled_labels, unrolled_predictions, average='binary')

    return {
        'sequence_f1': sequence_f1,
        'unrolled_sequence_f1': unrolled_sequence_f1
    }

In [224]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset
import torch

emotions = df['emotions'].explode().unique()
num_labels = len(emotions)
# Initialize the tokenizer
model_card = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_card)

# Initialize MultiLabelBinarizer for emotions
mlb = MultiLabelBinarizer()
mlb.fit(train_data['emotions'])

def preprocess_data(data, tokenizer, mlb):
    # Tokenize the utterances
    tokenized_inputs = tokenizer(
        [" ".join(utterance) for utterance in data['utterances']],
        padding='max_length',
        truncation=True,
        max_length=128, # we might wanna change this to max_len !
        return_tensors='pt'
    )

    # Encode the emotions
    encoded_labels = mlb.transform(data['emotions'])
    # Convert labels to tensors
    labels_tensor = torch.tensor(encoded_labels, dtype=torch.float)
    # Ensure the encoded labels are the correct shape (this step is crucial to avoid the batch size mismatch)
    if labels_tensor.shape[0] != tokenized_inputs['input_ids'].shape[0]:
        raise ValueError(f"Number of examples {tokenized_inputs['input_ids'].shape[0]} does not match number of labels {labels_tensor.shape[0]}")

    return Dataset.from_dict({
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels': labels_tensor
    })

# Now call preprocess_data to process your train and validation data
train_dataset = preprocess_data(train_data, tokenizer, mlb)
val_dataset = preprocess_data(val_data, tokenizer, mlb)

print("Single train example:", train_dataset[0])
print("Single validation example:",val_dataset[0])

Single train example: {'input_ids': [101, 2036, 1045, 2001, 1996, 2391, 2711, 2006, 2026, 2194, 1005, 1055, 6653, 2013, 1996, 1047, 2140, 1011, 1019, 2000, 24665, 1011, 1020, 2291, 1012, 2017, 2442, 1005, 2310, 2018, 2115, 2398, 2440, 1012, 2008, 1045, 2106, 1012, 2008, 1045, 2106, 1012, 2061, 2292, 1005, 1055, 2831, 1037, 2210, 2978, 2055, 2115, 5704, 1012, 2026, 5704, 1029, 2035, 2157, 1012, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [221]:
val_dataset['input_ids'][0]

[[101,
  2852,
  1012,
  21500,
  3917,
  1010,
  2045,
  1005,
  1055,
  1037,
  2835,
  2058,
  2182,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [101,
  4067,
  2017,
  1010,
  2852,
  1012,
  8109,
  1010,
  2021,
  1045,
  1005,
  1049,
  2383,
  2026,
  6265,
  2012,
  2023,
  2795,
  1010,
  2182,
  1999,
  1996,
  2690,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [101,
  1045,
  1005,
  1049,
  2383,
  6265,
  2157,
  2182,
  1010,
  2007,
  2026,
  2204,
  2767,
  9558,
  1010,
  2065,
  2002,
  1005,
  2222,
  4133,
  2007,
  2033,
  1012,
  102,
 

In [153]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy = "epoch"
)

# Initialize the model with the number of emotion labels
num_labels = len(mlb.classes_)
model = BertForSequenceClassification.from_pretrained(model_card, num_labels=num_labels)
# Freeze the BERT embedding layer weights
for name, param in model.named_parameters():
    if 'classifier' not in name: # Layer names not containing 'classifier' will be frozen
        param.requires_grad = False

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)
# Train the model
trainer.train()
# Train the model
trainer.train()
results = trainer.evaluate()

print(results)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch,Training Loss,Validation Loss,Sequence F1,Unrolled Sequence F1
1,0.578000,0.581855,0.475428,0.743736


Epoch,Training Loss,Validation Loss,Sequence F1,Unrolled Sequence F1
1,0.491800,0.514826,0.512398,0.765638


{'eval_loss': 0.5148258209228516, 'eval_sequence_f1': 0.5123978547771176, 'eval_unrolled_sequence_f1': 0.7656380316930776, 'eval_runtime': 2.7564, 'eval_samples_per_second': 145.117, 'eval_steps_per_second': 2.54, 'epoch': 1.0}


In [158]:
test_prediction_info = trainer.predict(val_dataset)
#metrics = compute_metrics(test_prediction_info)
#print(test_prediction_info)
test_prediction_info.predictions[0],test_prediction_info.label_ids[0]

(array([ 2.447779  ,  2.1871235 ,  0.77851427, -1.3586662 , -0.32813776,
         1.0689657 , -1.5161765 ,  0.18191618], dtype=float32),
 array([1., 1., 0., 1., 1., 1., 0., 0.], dtype=float32))

## For two classifier heads, one common model


In [ ]:
!pip install datasets==2.13.2
!pip install accelerate -U

In [ ]:
from transformers import BertPreTrainedModel, BertModel
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.nn import BCEWithLogitsLoss, Linear
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset

class BertForMultiLabelSequenceClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels_emotions, num_labels_triggers):
        super().__init__(config)
        self.num_labels_emotions = num_labels_emotions
        self.num_labels_triggers = num_labels_triggers

        self.bert = BertModel(config)
        self.classifier_emotions = Linear(config.hidden_size, num_labels_emotions)
        self.classifier_triggers = Linear(config.hidden_size, num_labels_triggers)

        # You can initialize weights here if needed

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels_emotions=None,
        labels_triggers=None,
        return_dict=None,
    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        pooled_output = outputs[1]

        logits_emotions = self.classifier_emotions(pooled_output)
        logits_triggers = self.classifier_triggers(pooled_output)

        loss = None
        # Calculate loss only if both labels are provided
        if labels_emotions is not None and labels_triggers is not None:
            loss_fct = BCEWithLogitsLoss()
            loss_emotions = loss_fct(logits_emotions.view(-1, self.num_labels_emotions), labels_emotions.view(-1))
            loss_triggers = loss_fct(logits_triggers.view(-1, self.num_labels_triggers), labels_triggers.view(-1))
            loss = loss_emotions + loss_triggers

        # Adjust the return statement to match what your compute_loss method expects
        if not return_dict:
            outputs = (logits_emotions, logits_triggers)
            if loss is not None:
                outputs = (loss,) + outputs
        else:
            outputs = {'loss': loss, 'logits_emotions': logits_emotions, 'logits_triggers': logits_triggers}

        return outputs
        """
        if not return_dict:
            print(f"Logits Emotions: {logits_emotions.shape}, Logits Triggers: {logits_triggers.shape}")
            output = (logits_emotions, logits_triggers) if loss is None else (loss, logits_emotions, logits_triggers)
            return (loss, logits_emotions, logits_triggers) if loss is not None else (logits_emotions, logits_triggers)
        else:
            raise NotImplementedError("return_dict is set to True, but this model does not support it.")
            """

In [ ]:
# Initialize MultiLabelBinarizers
mlb_emotions = MultiLabelBinarizer()
mlb_triggers = MultiLabelBinarizer()
mlb_emotions.fit(train_data['emotions'])
mlb_triggers.fit(train_data['triggers'])

# Initialize the tokenizer
model_card = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_card)

In [ ]:
def preprocess_data(data, tokenizer, mlb_emotions, mlb_triggers):
    tokenized_inputs = tokenizer(
      [" ".join(utterance) for utterance in data['utterances']],
      padding='max_length',
      truncation=True,
      max_length=128,
      return_tensors='pt'
    )

    # Encode the emotions and triggers
    encoded_labels_emotions = mlb_emotions.transform(data['emotions'])
    encoded_labels_triggers = mlb_triggers.transform(data['triggers'])

    # Convert labels to tensors and reshape
    labels_emotions = torch.tensor(encoded_labels_emotions, dtype=torch.float).view(-1, mlb_emotions.classes_.size)
    labels_triggers = torch.tensor(encoded_labels_triggers, dtype=torch.float).view(-1, mlb_triggers.classes_.size)

    return Dataset.from_dict({
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels_emotions': labels_emotions,
        'labels_triggers': labels_triggers
    })

# Then preprocess the training and validation data
train_dataset = preprocess_data(train_data, tokenizer, mlb_emotions, mlb_triggers)
val_dataset = preprocess_data(val_data, tokenizer, mlb_emotions, mlb_triggers)

print("Single train example:", train_dataset[0])
print("Single validation example:",val_dataset[0])


In [ ]:
from sklearn.metrics import f1_score

# Define your compute_metrics function to handle both sets of labels
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits_emotions = logits[0]
    logits_triggers = logits[1]
    labels_emotions = labels[0]
    labels_triggers = labels[1]
    # Sigmoid function to convert logits to probabilities
    sigmoid = lambda x: 1 / (1 + np.exp(-x))
    # Threshold to convert probabilities to binary predictions
    threshold = 0.5

    # Convert logits to probabilities
    probs_emotions = sigmoid(logits_emotions)
    probs_triggers = sigmoid(logits_triggers)

    # Convert probabilities to binary predictions
    preds_emotions = (probs_emotions > threshold).astype(int)
    preds_triggers = (probs_triggers > threshold).astype(int)

    # Calculate the f1-score for emotions
    f1_emotions = f1_score(labels_emotions, preds_emotions, average='macro')
    # Calculate the f1-score for triggers
    f1_triggers = f1_score(labels_triggers, preds_triggers, average='macro')

    # To calculate sequence-level metrics, we might need to modify data processing

    # to include sequence IDs, and then group by sequence ID before calculating metrics.
    return {
        'f1_emotions': f1_emotions,
        'f1_triggers': f1_triggers,
    }

In [ ]:
#TODO make this compatible with model class by changing the way the absence of loss is handled, similar to CustomTrainer4

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Unpack inputs
        labels_emotions = inputs.pop("labels_emotions", None)
        labels_triggers = inputs.pop("labels_triggers", None)

        # Forward pass
        outputs = model(**inputs)

        # If loss is not included in outputs, calculate it using the logits
        if len(outputs) == 2:
            logits_emotions, logits_triggers = outputs
            loss_fct = BCEWithLogitsLoss()
            if labels_emotions is not None and labels_triggers is not None:
                loss_emotions = loss_fct(logits_emotions.view(-1, self.model.num_labels_emotions), labels_emotions.view(-1, self.model.num_labels_emotions))
                loss_triggers = loss_fct(logits_triggers.view(-1, self.model.num_labels_triggers), labels_triggers.view(-1, self.model.num_labels_triggers))
                loss = loss_emotions + loss_triggers
        elif len(outputs) == 3:
            # Outputs include a loss
            loss, logits_emotions, logits_triggers = outputs
        else:
            raise ValueError(f"Unexpected number of outputs from model: {len(outputs)}")

        # If loss was not computed within the model, we compute it based on the logits and the labels
        if loss is None and labels_emotions is not None and labels_triggers is not None:
            loss_fct = BCEWithLogitsLoss()
            loss_emotions = loss_fct(logits_emotions.view(-1, self.model.num_labels_emotions), labels_emotions.view(-1))
            loss_triggers = loss_fct(logits_triggers.view(-1, self.model.num_labels_triggers), labels_triggers.view(-1))
            loss = loss_emotions + loss_triggers

        return (loss, outputs) if return_outputs else loss


In [ ]:
class CustomTrainer4(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Unpack inputs
        labels_emotions = inputs.pop("labels_emotions", None)
        labels_triggers = inputs.pop("labels_triggers", None)

        # Forward pass
        outputs = model(**inputs)

        # If loss is not included in outputs, calculate it using the logits
        if 'loss' not in outputs:
            logits_emotions = outputs['logits_emotions']
            logits_triggers = outputs['logits_triggers']
            loss_fct = BCEWithLogitsLoss()
            if labels_emotions is not None:
                loss_emotions = loss_fct(logits_emotions.view(-1, self.model.num_labels_emotions), labels_emotions.view(-1))
            if labels_triggers is not None:
                loss_triggers = loss_fct(logits_triggers.view(-1, self.model.num_labels_triggers), labels_triggers.view(-1))
            loss = loss_emotions + loss_triggers
            outputs['loss'] = loss
        else:
            # Loss is included in outputs
            loss = outputs['loss']

        return (loss, outputs) if return_outputs else loss


In [ ]:
# Initialize model with the number of emotion and trigger labels
num_labels_emotions = len(mlb_emotions.classes_)
num_labels_triggers = len(mlb_triggers.classes_)
model = BertForMultiLabelSequenceClassification.from_pretrained(
    model_card,
    num_labels_emotions=num_labels_emotions,
    num_labels_triggers=num_labels_triggers
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Assuming train_dataset is a Hugging Face dataset with the correct columns
example = train_dataset[0]

# Now, before passing tensors to the model, ensure they are also on the same device
example_tensors = {k: torch.tensor([v]).to(device) for k, v in example.items()}

# Call the model
with torch.no_grad():
    outputs = model(**example_tensors)
    print(outputs)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy = "epoch"
    report_to = ""
)

# # Initialize the model with the number of emotion labels
# num_labels = len(mlb.classes_)
# model = BertForSequenceClassification.from_pretrained(model_card,
#                                                       num_labels=num_labels,
#                                                        )

# Freeze the BERT embedding layer weights
for name, param in model.named_parameters():
    if 'classifier' not in name: # Layer names not containing 'classifier' will be frozen
        param.requires_grad = False

# Initialize the Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)

# Train the model
trainer.train()
results = trainer.evaluate()

print(results)
